# .SPE Read/Analyse

- __version__ = 3.00
- __author__ = Manan Shah


--------------------------------------------- **Nomenclature** ---------------------------------------------

- aa; bb; cc;.... # counters
- ClassName
- Exception_Name
- function_name
- function_parameter_name
- GLOBAL_CONSTANT_NAME
- global_Var_Name
- instance_var_name
- local_var_name
- _method_name
- module_name
- package_name

## Load libraries

In [ ]:
import tkinter as tk  # A robust and platform independent windowing toolkit to implement GUI.
import numpy as np
import pandas as pd

# from IPython.display import SVG
# from IPython.display import Image as IM
from IPython.display import Markdown as MD
from IPython.display import display
from pathlib import Path
from tkinter.filedialog import askopenfilenames, askopenfiles
from pandas import DataFrame as DF
from pint import UnitRegistry  # Assign units to qunatities

# u = UnitRegistry(auto_reduce_dimensions=True) right syntax but not sure why not needed!
u = UnitRegistry()  
# "~": abbreviation of quantity (e.g. meter -> m) and "P": Physical quantity format (e.g. meter / second**2 -> meter/second^2)
u.default_format = "~P"
Q_ = u.Quantity

from scipy import ndimage, signal
from scipy.ndimage import (
    convolve,
    gaussian_filter,
    gaussian_filter1d,
    gaussian_laplace,
    maximum,
    maximum_filter,
    maximum_filter1d,
    maximum_position,
    median_filter,
    minimum,
    minimum_filter,
    minimum_filter1d,
    minimum_position,
    percentile_filter,
    standard_deviation,
    sum,
    uniform_filter,
    uniform_filter1d,
    variance,
)

from scipy.signal import (
    convolve,
    detrend,
    filtfilt,
    find_peaks,
    lfilter,
    medfilt,
    medfilt2d,
    order_filter,
    peak_prominences,
    peak_widths,
    resample,
    resample_poly,
    savgol_filter,
    sosfilt,
    symiirorder1,
    symiirorder2,
    upfirdn,
    wiener,
)

## User defined functions

### Format display text

In [ ]:
def printmd(string, color=None):
    """
    Displays the string in color with markdown effects.

        Parameters:
            string (str): "The string to be printed."
            color (str): "Color of the string (e.g., "green", "red")."

        Display:
            display(MD(colorstr)): A colored string with markdown effects.

        Example: printmd(f"**the value: {a}**", color = "green")
    """

    colorstr = "<span style='color:{}'>{}</span>".format(
        color,
        string,
    )
    display(MD(colorstr))

### Import files

In [ ]:
def import_files(ftype, fextension, dialogue_title):
    """
    Import files of a specific type via a dialogue box.

        Parameters:
           ftype (str): File type (e.g. .SPE Files).
           fextension (str): File extension (e.g. ".*spe").
           dialogue_title (str): Title of the dialogue box (e.g. "Import files").

        Functions:
            askopenfilenames(parent=root, filetypes=[(ftype, fextension,)], title=dialogue_title): Creates a modal, native look-and-feel dialog, wait for the user’s selection, then return the selected filename(s) that correspond to existing file(s). 
            pathlib.Path(): Offers classes representing filesystem paths with semantics appropriate for different operating systems.

        Returns:
            file_names: file_names (file objects) are fed to self.fnames in class File where they'll be read one by one.)

        Example: import_files(self, ftype = ".SPE files", fextension = ".*spe", dialogue_title = "Import .SPE files")

        Pattern | Meaning
           *    | matches everything
           ?    | matches any single character
         [seq]  | matches any character in seq
         [!seq] | matches any character not in seq

         For a literal match, wrap the meta-characters in brackets. For example, '[?]' matches the character '?'
    """

    root = tk.Tk()
    file_names = askopenfilenames(
        parent=root,
        filetypes=[
            (
                ftype,
                fextension,
            )
        ],
        title=dialogue_title,
    )
    root.destroy()  # Destroy the window after the action is taken.

    # .resolve() resolves the symlinks and eliminate “..” components to walk an arbitrary filesystem path upwards.
    file_full_path = Path(file_names[0]).resolve()  
    importDir = file_full_path.parent
    printmd(f"**Import directory : {importDir}**", "orange")
    printmd(f"**No. of file(s) selected : {len(file_names)}**\n", "orange")
    display(MD("**File(s) information**"))

    return file_names

### Read from a file

In [ ]:
def read_at(file, position, size, dtype):
    """
    Returns data from a text or binary file starting at a particular position for a given size and specified data-type.

        Parameters:
           file (file object): Opens a single file object or a filename. (e.g. here .SPE file in a binary mode).
           position (int): move the reading cursor characters ahead.
           size (int): Number of items to read. -1 means all items (i.e., the complete file).
           dtype (data type object): A data type object (an instance of numpy.dtype class) describes how the bytes in the                fixed-size block of memory corresponding to an array item should be interpreted. It describes the following                  aspects of the data:
               1. Type of the data (integer, float, Python object, etc.)
               2. Size of the data (how many bytes is in e.g. the integer)
               3. Byte order of the data (little-endian or big-endian)
               4. If the data type is structured data type, an aggregate of other data types (e.g. describing an array item                     consisting of an integer and a float).

        Functions:
            file.seek(position): Set the current read/write position. (e.g. file.seek(offset = position, from = 0 (0 =                                        beginning, 1 = current, 2 = end)))
            np.fromfile(file, dtype, size):  A highly efficient way of reading binary data with a known data-type
            
            NOTE: Do not rely on the combination of tofile and fromfile for data storage, as the binary files generated are               not platform independent. In particular, no byte-order or data-type information is saved. Data can be stored in               the platform independent .npy format using save and load instead.

        Returns:
            np.fromfile(file, dtype, size): Construct an array from data in a text or binary file.

        Example: read_at(file_obj, position = 23, size = 8, ntype = np.float32)
    """
    
    file.seek(position)  # set the current read/write position.
    return np.fromfile(file, dtype, size)

### Data Reverse

In [ ]:
def data_reverse(data):
    """
    Returns an ndarray reversed from left to right to the original one. This funcation accepts only an array with >= 2D. In       order to do so, all the X- axis and data arrays are transformed into a matrix with atleast the shape of (1, xPixels).

    Parameters:
        data (ndarray): atleast 2D array.

    Functions:
        np.empty(shape, dtype=float, order='C'): Return a new array of given shape and type, without initializing entries.
        np.fliplr(m {Input array, must be at least 2-D.}): Flip the entries in each row in the left/right direction.
                                                           Columns are preserved, but appear in a different order than                                                                  before.

    Returns:
        data_reverse (ndarray): ndarray  with the same shape as of the input ndarray.
    """

    index = 0
    data_reverse = np.empty(shape=np.shape(data))

    for aa in data:
        data_reverse[index] = np.fliplr(aa)
        index += 1

    if np.shape(data) == np.shape(data_reverse):
        printmd(
            f"The data is reversed and returned as {type(data_reverse)} with the same shape of {np.shape(data_reverse)}."
        )
        return data_reverse

    else:
        raise Exception(
            "The shape of data and reversed data does not match! Pls. make sure that the input- data has at least 2D."
        )

### Region selection and Binning

In [ ]:
def region_bin(data, no_regns, stpts, endpts):
    """
    Returns (2D) selected regions of a parent matrix as well as the binned 1D matrices of the corresponding selected regions.
    It accepts the start/end points as a list.

    NOTE: The shape of data_regn should be (no_regns,) because the selected regions might have diferent matrix size.

    Parameters:
        data (list, ndarray): A 2D array from which an inerested region is going to be selected.
        no_regions (int): A positive integer to select the number of regions from the given 2D matrix.
        stpts (list): A list of starting points for interested regions.
        endpts (list): A list of ending points for interested regions.

    Returns:
        data_regn (list): List of ndarrays of sliced regions.
        data_regn_binned (ndarray): ndarray of binned corresponding sliced regions.
    """

    if no_regns == len(stpts) == len(endpts) or np.shape(data) > (1, 1):
        printmd(f"No. of regions selected = {no_regns}")

        data_regn = []
        data_regn_binned = []

        for aa in range(no_regns):
            data_regn.append(data[stpts[aa] : endpts[aa]])
            printmd("\n")
            printmd(f"Region {aa} [Start, End) = [{stpts[aa]} : {endpts[aa]})")
            printmd(f"Shape of region {aa} = {np.shape(data_regn[aa])}")

            data_regn_bin = np.reshape(
                data_regn[aa].sum(axis=0) / (endpts[aa] - stpts[aa]),
                (1, np.shape(data_regn[aa])[1]),
            )
            data_regn_binned.append(data_regn_bin)
            printmd(f"Shape of binned region {aa} = {np.shape(data_regn_binned[aa])}")

        return data_regn, np.asanyarray(data_regn_binned)

    else:
        raise Exception(
            "The no. of regions and corresponding no. of start/end points do not match! Or the selected data is not 2D!"
        )

### Differentiation

In [ ]:
def differentiation(data, x):
    """
    Returns a list of derivatetive of data w.r.t "x". This funcation accepts only an array with >= 2-Dimnesions.

    Parameters:
        data (ndarray): atleast 2D array.
        x (ndarray): atleast 2D array.:

    Returns:
        derivative (list): List of ndarray(s) with the shape of the input list of ndarray(s) - no. of rows.
    """

    dy = []
    dx = []
    derivative = []

    if np.shape(data[0])[1] == np.shape(x[0])[1]:
        for aa in range(len(data)):
            dy.append(np.diff(data[aa]))  # dy
            dx.append(np.diff(x[aa]))  # dx
            derivative.append(np.divide(dy[aa], dx[aa])            )
            printmd(f"Shape of derivated data{aa} = {np.shape(derivative[aa])}")

        printmd(f"The differentiation of the data w.r.t. 'x' is done.")
        return derivative

    else:
        raise Exception(
            f"The shape of data {np.shape(data[0])} and 'x' {np.shape(x[0])} does not match!"
        )

### Reflection contrast

In [ ]:
def reflection_contrast(sam_data, sub_data):
    """
    Returns the reflection contrast from selected (binned) regions.

    Parameters:
        sam_data (ndarray): An array of reflection intensity from sample.
        sub_data (ndarray): An array of reflection intensity from substrate.

    Returns:
        reflection_contrast1 (list): list of (array(s) of) reflection contrast calucated using methode 1 (or 2).
    """

    if np.shape(sam_data[0]) == np.shape(sub_data[0]):
        printmd(f"No. of (binned) regions selected = {len(sam_data)}")

        reflection_contrast1 = []
        reflection_contrast2 = []

        for aa in range(len(sam_data)):
            reflection_contrast1.append((sam_data[aa] - sub_data[aa]) / sub_data[aa]) 
            printmd(f"Shape of RC region {aa} = {np.shape(reflection_contrast1[aa])}")

        return reflection_contrast1  # or reflection_contrast2

    else:
        raise Exception(
            f"The shape of sample {np.shape(sam_data[0])} and substrate {np.shape(sub_data[0])} does not match!"
        )

### Filters

In [ ]:
def mvg_window_fltr(data, window_type, mode, **kwargs):
    """
    Returns filtered data by convoluting input data with a chosen moving window.

    Parameters:
        data (ndarray): input ndarrray vector. It must be a 2D array.
        window_type (tupple with respective arguments): 'Window name' (string) tuppled with 'positional argument(s)'                                                                  respective to that particular window.
        mode (string {optional}): (reflect, constant, nearest, mirror, wrap); The mode parameter determines how the input                                       array is extended beyond its boundaries. Default is ‘reflect’. Behavior for each valid                                       value is as follows:
                                  ‘reflect’ (d c b a | a b c d | d c b a) The input is extended by reflecting about the edge                                   of the last pixel.
                                  ‘constant’ (k k k k | a b c d | k k k k) The input is extended by filling all values beyond                                    the edge with the same constant value, defined by the cval parameter.
                                  ‘nearest’ (a a a a | a b c d | d d d d) The input is extended by replicating the last                                          pixel.
                                  ‘mirror’ (d c b | a b c d | c b a) The input is extended by reflecting about the center of                                    the last pixel.
                                  ‘wrap’ (a b c d | a b c d | a b c d) The input is extended by wrapping around to the                                          opposite edge.
        cval (scalar {optional}): Value to fill past edges of input if mode is ‘constant’. Default is 0.
        origin (int or sequence {optional}): Controls the placement of the filter on the input array’s pixels. A value of 0                                                (default) centers the filter over the pixel, with positive values shifting the                                                filter to the left, and negative ones to the right. By passing a sequence of                                                  origins with length equal to the number of dimensions of the input array,                                                    different shifts can be specified along each axis.

    Returns:
        data_fltrd (ndarray): The convolved array of the input array.
    """

    """    
        # Original window dictionary ffrom scipy.signal.windows souce code
        windowsAll_dict = {  
            ('barthann', 'brthan', 'bth'): signal.barthann,
            ('bartlett', 'bart', 'brt'): signal.bartlett, 
            ('blackman', 'black', 'blk'): (signal.blackman), 
            ('blackmanharris', 'blackharr', 'bkh'): signal.blackmanharris, 
            ('bohman', 'bman', 'bmn'): signal.bohman, 
            ('boxcar', 'box', 'ones', 'rect', 'rectangular', 'mvg avg', 'mvg_avg', 'uniform', 'flat'): signal.boxcar, 
            ('chebwin', 'cheb'): signal.chebwin, 
            ('cosine', 'halfcosine'): signal.cosine,
            ('exponential', 'poisson'): signal.exponential, 
            ('flattop', 'flat', 'flt'): (signal.flattop), 
            ('gaussian', 'gauss', 'gss'): signal.gaussian, 
            ('general cosine', 'general_cosine'): signal.windows.general_cosine,
            ('general gaussian', 'general_gaussian', 'general gauss', 'general_gauss', 'ggs'): signal.general_gaussian, 
            ('general hamming', 'general_hamming'): signal.windows.general_hamming,
            ('hamming', 'hamm', 'ham'): signal.hamming, 
            ('hanning', 'hann', 'han'): signal.hann, 
            ('kaiser', 'ksr'): signal.kaiser, 
            ('medfilt', 'median filt'): signal.medfilt, 
            ('nuttall', 'nutl', 'nut'): signal.nuttall, 
            ('parzen', 'parz', 'par'): signal.parzen, 
            ('savgol coeffs' ,'savgol_coeffs'): signal.savgol_coeffs,
            ('slepian', 'slep', 'optimal', 'dpss', 'dss'): signal.slepian, 
            ('triangle', 'triang', 'tri'): signal.triang,  
            ('tukey', 'tuk'): signal.tukey,  
            (): ndimage.uniform_filter1d} 
    """

    windowsAll_tuple = (
        (("barthann", "brthan", "bth"), signal.barthann),
        (("bartlett", "bart", "brt"), signal.bartlett),
        (("blackman", "black", "blk"), signal.blackman),
        (("blackmanharris", "blackharr", "bkh"), signal.blackmanharris),
        (("bohman", "bman", "bmn"), signal.bohman),
        (
            (
                "boxcar",
                "box",
                "ones",
                "rect",
                "rectangular",
                "mvg avg",
                "mvg_avg",
                "uniform",
                "flat",
                "avg",
            ),
            signal.boxcar,
        ),
        (("chebwin", "cheb"), signal.chebwin),
        (("cosine", "halfcosine"), signal.cosine),
        (("exponential", "poisson"), signal.exponential),
        (("flattop", "flt"), signal.flattop),
        (("gaussian", "gauss", "gss"), signal.gaussian),
        (("general cosine", "general_cosine"), signal.windows.general_cosine),
        (
            (
                "general gaussian",
                "general_gaussian",
                "general gauss",
                "general_gauss",
                "ggs",
            ),
            signal.general_gaussian,
        ),
        (("general hamming", "general_hamming"), signal.windows.general_hamming),
        (("hamming", "hamm", "ham"), signal.hamming),
        (("hanning", "hann", "han"), signal.hann),
        (("kaiser", "ksr"), signal.kaiser),
        (("medfilt", "median filt"), signal.medfilt),
        (("nuttall", "nutl", "nut"), signal.nuttall),
        (("parzen", "parz", "par"), signal.parzen),
        (("savgol coeffs", "savgol_coeffs"), signal.savgol_coeffs),
        (("slepian", "slep", "optimal", "dpss", "dss"), signal.slepian),
        (("triangle", "triang", "tri"), signal.triang),
        (("tukey", "tuk"), signal.tukey),
    )

    # convert tupples into a modified dictionary where tuppled keys are untuppled and correspond to the same value. e.g. {'a': 1, 'b':1, 'c':2} instead of {(a,b):1, ('c'):2}
    windowsAll_dict_mod = {
        key: value for keys, value in windowsAll_tuple for key in keys
    }

    if data.ndim == 2:
        printmd(f"Input vector shape: {np.shape(data)}")

    else:
        raise Exception(
            f"The data {np.shape(data[0])} should be comprise of either 1D matrix or 2D array."
        )

    if isinstance(window_type, tuple):
        window_str = window_type[0]
        args = window_type[1:]

        # Returns a list containing the dictionary's keys
        if window_str in windowsAll_dict_mod.keys():
            printmd(f"Filter window type: {window_str}.")
            printmd(f"{len(args)} additional argument(s): {args}")

        else:
            while True:
                try:
                    window_type = eval(
                        input(
                            f"Pls. enter the window type (e.g. ('general_gaussian', *args)): "
                        )
                    )  # Evaluate string as a tuple.
                    window_str = window_type[0]
                    args = window_type[1:]

                    if (window_str in windowsAll_dict_mod.keys()) and len(window_type) > 1:
                        printmd("length:", len(window_type))
                        printmd(f"Entered window type: {window_str}")
                        printmd(f"Entered {len(args)} additional argument(s): {args}")

                    else:
                        raise ValueError

                except ValueError:
                    printmd(
                        f"Entered {window_str} window does not match with the available window types. Available windows are: {windows_all_mod}"
                    )
                    continue

                else:
                    break

        wind_func = windowsAll_dict_mod.get(
            window_str
        )  # Returns the value of the specified key
        w = wind_func(*args)  # Window wieghts
        norm_window = w / sum(w)

        printmd(f"Window: {norm_window}")

    else:
        raise Exception(
            "Please enter a window_type and it's parameters in a tuple form (e.g. ('general_gaussian, 1, 7) or ('flat', 5)'."
        )

    if round(sum(norm_window), 2) == 1:
        printmd(f"Sum of window elements: {sum(norm_window)}")

        data_fltrd = []
        for aa in range(len(data)):
            data_fltrd.append(
                ndimage.convolve(input=data[aa], weights=norm_window, mode=mode)
            )

        if np.shape(data) == np.shape(np.asanyarray(data_fltrd)):
            printmd(
                f"The data is smoothed with a normalized window and returned with same shape of {np.shape(np.asanyarray(data_fltrd))}"
            )
            return np.asanyarray(data_fltrd)
        else:
            raise Exception(
                f"The shape of input data {np.shape(data)} does not match with the filtered data {np.shape(np.asanyarray(data_fltrd))}."
            )

    else:
        raise Exception(
            f"Them sum of the window elements is {sum(norm_window)} but it should be '1'."
        )

### File close

In [ ]:
def file_close(files):
    """
    Closes file(s) when 'with open() as file_object' is not used.

    Parameter:
        files (list (file object)): A list of file object(s) to be closed.
    """

    n_files = len(files)

    for file in files:  # Close each open file.
        file.close()

    if n_files > 1:
        printmd(f"**The {n_files} imported files are closed.**", "green")

    else:
        printmd(f"**The imported file is closed.**", "green")

### Load .SPE files

In [ ]:
class File:

    """
    This class deals with importing only .SPE files and displaying the relevant information. The relevant data is shown for       each file while the metadata is only shown once; since, it's a standard experimental practice to keep most of the             parameters same for the same kind of experiment. It also raises appropriate errors or pass alert messages when               inconstistancy is found in data or metadata. The most of the parameters are extracted from the .SPE file header; however     some parameters such as experiment temp., pressure, incident power, and source wavelength are extracted from the file         name. Therefor, please use the prefered file naming convention for optimum results.

    Use Defined Functions:
        import_files(ftype, fextension, dialogue_title)
        printmd(string, color)
        read_at(file, position, size, dtype)

    Methods:
        _load_spe_files(self,)

        _spe_read(self, file): Feed .spe file in a loop.

            Paramers:
                file (binary file object): Feed a binary file object using an "open()" function.

            Methods:
                # ---------- from file name ---------- #
                _step_glue()
                _experiment_temp()
                _excitation_source()
                _incident_power()

                # ---------- from file ---------- #
                _exposure_time()
                _accumulations()
                _data_dimension()
                _no_frames()
                _axes()
                _experiment_data()
                _pressure()
                _grating()

            Misc. variables:
                self.wl_flag
                self.fname_corr_flag

            Returns:
                spedict (dictonary): Contains information about the experimental conditions
                     and data of a file extracted using the above methods.

        _standard_info(self,)
            Methods:
                # ---------- from file ---------- #
                _no_frames()
                _detector_temp()
                _intensified_gain()
                _versions()
                _data_corrections()
                _messege()

    Returns: Displays information about the experimental conditions which should be the constant for all the experiments.

        NOTE: The methods can be called upon as class_obj.method().nested_method()
    """

    ###################### Header offsets (gobally available within this class) (GLOBAL_CONSTANT_NAME) ######################
    ACCUMULATIONS1 = 668  # When accumulations are < 32767
    ACCUMULATIONS2 = 1422  # When accumulations are > 32767
    BG_CORRECTION = 150  # 1 if background subtraction done
    DATA_START = 4100
    CONTROLLER_VERSION = 0
    COSMIC_CORRECTION = 1438  # Flag
    COSMIC_CORR_TYPE = 1440
    COSMIC_THRESHOLD = 1442
    DETECTOR_TEMP = 36
    EXPOSURE_TIME = 10
    EXPERIMENT_DATE = 20
    EXPERIMENT_TIME_LOCAL = 172
    #     FILE_COMMENTS = 200
    FLATFIELD_CORRECTION = 706  # 1 if flat fiels is applied
    #     GAIN = 198  # NO idea!
    GRATING_GROOVES = 650
    HEADER_VERSION = 1992
    NUM_FRAMES = 1446
    NUM_ROI = 1488  # or 1510; dtype = short Still not working.
    PIMAX_GAIN = 148
    READOUT_TIME = 672
    SOFTWARE_PACKAGE = 1508  # Software package created this file
    SOFTWARE_VERSION = 688  # Version of SW creating this file.
    STEP_GLUE_FLAG = 76
    #     WAVELENGTH_CENTER = 72  # Not working for Step&Glue mode
    #     WAVELENGTH_END = 82  # Only working for Step&Glue mode
    WAVELENGTH_OVERLAP = 86
    #     WAVELENGTH_START = 78  # Only working for Step&Glue mode
    WAVELENGTH_RESOLUTION = 90  # Only working for Step&Glue mode
    X_PIXELS = 42
    Y_PIXELS = 656

    ###################### Start of X Calibration Structure (3000 - 3488) ######################
    X_CALIBRATION_VALID = 3098  # flag if calibration is valid
    X_LASER_POSITION = 3311
    #     X_OFFSET = 3000  # offset for absolute data scaling
    X_POLYNOMIAL_ORDER = 3101  # ORDER of calibration POLYNOM
    X_POLYNOMIAL_COEFFICIENT = 3263  # polynom COEFFICIENTS

    ###################### Start of Y Calibration Structure (3489 - 3977) ######################
    Y_CALIBRATION_VALID = 3587  # flag if calibration is valid
    #     Y_OFFSET = 3489  # offset for absolute data scaling
    Y_POLYNOMIAL_ORDER = 3590  # ORDER of calibration POLYNOM
    Y_POLYNOMIAL_COEFFICIENT = 3752  # polynom COEFFICIENTS

    # -------------------------------------------------------------------------------------------------------------------#

    # self (instance of a class) gives access to use the attributes and methods of the class. It binds the attributes with the given arguments.
    def __init__(
        self,
    ):
        """__init__ method’s docstring is inaccurate — it just initializes the object to its factory-default settings after its creation."""
        self._load_spe_files()

    # ----------------------------------------- Method to import only .SPE files! -----------------------------------------#
    def _load_spe_files(
        self,
    ):

        try:
            self.fnames = import_files(
                ftype=".SPE files",
                fextension=".*spe",
                dialogue_title="Import .SPE files",
            )

        except IndexError:
            raise LookupError("Please select at least one .SPE file to proceed!")

    # ---------------------------------------- Method to read imported .SPE file(s) ----------------------------------------#
    def _spe_read(self, file):
        """
        Reads the data, the header (other vital info.), and the file name from the imported .SPE file.
        The extracted data is then assigned to a dictionary (spedict) and dipöayed as the dataframe.
        """

        full_path = Path(file).resolve()

        ####################### Flags #######################
        self.wl_flag = 0
        self.fname_corr_flag = 0
        # ----------------------------------------------------

        self.name = full_path.stem
        # self.nameCorrect0 = self.name.replace("-", "_")  # string.replace(old, new, count) iff old values is found!
        self.nameCorrect0 = self.name
        self.nameCorrect1 = self.nameCorrect0.replace(
            " ",
            "_",
        )
        self.nameCorrect2 = self.nameCorrect1.lower()
        self.nameCorrect3 = self.nameCorrect2.replace(
            "milibar",
            "mbar",
        )
        self.nameCorrect4 = self.nameCorrect3.replace(
            "mb",
            "mbar",
        )
        self.nameCorrect5 = self.nameCorrect4.replace(
            "µw",
            "uw",
        )

        self.name_split_list = self.nameCorrect1.split(
            "_"
        )  # Split chuncks using "_" delimiter.
        self.name_split_list1 = self.nameCorrect5.split("_")

        # --------------- The following info is not stored in a .SPE header. It is extracted from a file name. ------------#
        def _experiment_temp():
            """
            Returns the temperature at which the experiment was performed.
            The temperature value is extracted from the file name (if extractable!),
            otherwise asks the user to  input it manually.
            """

            if "RT" in self.nameCorrect1:
                self.temperature = 293 * u.degK

            elif "K" in self.nameCorrect1:
                subString = "K"
                result = [aa for aa in self.name_split_list if subString in aa]
                self.temperature = int(result[0][0:-1]) * u.degK

                if self.temperature == "K":
                    temp = self.name_split_list.index("K")
                    self.temperature = (
                        int(self.name_split_list[temp - 1]) * u.degK
                    )  # + self.name_split_list[temp]

                else:
                    pass 
                  
            else:
                print("\n")
                printmd(
                    f"Temp. info is either mising or could not be extracted from {self.name}.",
                    "yellow",
                )
                self.fname_corr_flag = 1

                while True:
                    try:
                        self.temperature = (
                            int(input(f"Pls. enter temp. (e.g. xx): ")) * u.degK
                        )

                        if self.temperature < 0:
                            raise ValueError
                        else:
                            pass

                    except ValueError:
                        printmd("Pls. enter a positive integer value only!", "red")
                        continue

                    else:
                        break

            return self.temperature
            self.fname_corr_flag = 1

        def _excitation_source():
            """
            Returns the excitation source wavelength used for the given experiment.
            The wavelength is extracted from file name (if extractable!), otherwise asks the user to input it manually.
            """

            if "nm" in self.nameCorrect1 and (
                "cw" in self.nameCorrect5 or "pulse" in self.nameCorrect5
            ):

                if "cw" in self.nameCorrect5:
                    subString = "nm"
                    result = [aa for aa in self.name_split_list1 if subString in aa]
                    self.ext_source = (
                        int(result[0][0:-6]) * u.nm
                    )  # strip nm(cw) (last 6 characacters)

                    if result[0] == "nm":
                        wvlngth = self.name_split_list.index("nm")
                        self.ext_source = int(self.name_split_list[wvlngth - 1]) * u.nm
                        self.ext_source_string = f"{self.ext_source} (CW)"

                    else:
                        pass

                    self.ext_source_string = f"{self.ext_source} (CW)"

                elif "pulse" in self.nameCorrect5:
                    subString = "nm"
                    result = [aa for aa in self.name_split_list1 if subString in aa]
                    self.ext_source = (
                        int(result[0][0:-9]) * u.nm
                    )  # strip nm(pulse) (last 9 characacters)

                    if result[0] == "nm":
                        wvlngth = self.name_split_list.index("nm")
                        self.ext_source = int(self.name_split_list[wvlngth - 1]) * u.nm
                        self.ext_source_string = f"{self.ext_source} (pulsed)"

                    else:
                        pass

                    self.ext_source_string = f"{self.ext_source} (pulsed)"

                else:
                    pass

            elif (
                "wl" in self.nameCorrect5
                or "refl" in self.nameCorrect5
                or "rfl" in self.nameCorrect5
                or "cnt" in self.nameCorrect5
            ):
                self.ext_source_string = "White light"
                self.wl_flag = 1

            else:
                print("\n")
                printmd(
                    f"The excitation wavelength info is either mising or could not be extracted from {self.name}.",
                    "yellow",
                )

                while True:
                    try:
                        self.ext_source_string = input(
                            "Pls. enter excitation wavelength (e.g. xxx nm (CW/pulsed) or white light (wl)): "
                        )
                        if (
                            self.ext_source_string == "wl"
                            or self.ext_source_string == "white light"
                        ):
                            self.ext_source_string = "White light"
                            self.wl_flag = True
                            break

                        elif (
                            "CW" in self.ext_source_string
                            or "cw" in self.ext_source_string
                        ):
                            # Find numerical values from the string and store them in a list.
                            self.ext_source = [
                                int(aa)
                                for aa in self.ext_source_string.split()
                                if aa.isdigit()
                            ]
                            self.ext_source = self.ext_source[0] * u.nm
                            self.ext_source_string = f"{self.ext_source} (CW)"
                            break

                        elif "pulse" in self.ext_source_string:
                            # Find numerical values from the string and store them in a list.
                            self.ext_source = [
                                int(aa)
                                for aa in self.ext_source_string.split()
                                if aa.isdigit()
                            ]
                            self.ext_source = self.ext_source[0] * u.nm
                            self.ext_source_string = f"{self.ext_source} (pulsed)"
                            break

                        else:
                            raise ValueError  # If a value is missing then raise the value error.

                    except ValueError:
                        printmd("CW or pulsed was missing!", "red")
                        continue

                    else:
                        break

            return self.ext_source_string
            self.fname_corr_flag = 1

        def _incident_power():
            """
            Returns the incident power used for the given experiment. The incident power is extracted from the file name (if             extractable!), otherwise asks the user to  input it manually. It is prefered to measure the incident power right             before the sample.

            NOTE: Generally, for CW laser source the incident power reading is correct with normal diode based power meters.
            However, for pulsed lasers, one should use the pulsed power meter to get the 'rms'/'aveage' power with pick                   power. Usually in the data analysis the power density ([incident power]/[laser spot area]) is a useful quantity               rather than just the incident power.
            """

            if "uw" in self.nameCorrect5:
                subString = "uw"
                result = [aa for aa in self.name_split_list1 if subString in aa]
                self.power = float(result[0][0:-2]) * u.microwatt

                if result[0] == "uw":
                    power_index = self.name_split_list1.index("uw")
                    self.power = (
                        float(self.name_split_list1[power_index - 1]) * u.microwatt
                    )

                else:
                    pass

            elif "mw" in self.nameCorrect2:
                subString = "mw"
                result = [aa for aa in self.name_split_list1 if subString in aa]
                self.power = float(result[0][0:-2]) * u.milliwatt

                if result[0] == "mw":
                    power_index = self.name_split_list1.index("mw")
                    self.power = (
                        float(self.name_split_list1[power_index - 1]) * u.milliwatt
                    )

                else:
                    pass

            elif self.wl_flag == 1:
                self.power = "na"

            else:
                print("\n")
                printmd(
                    f"Incident power info is either mising or could not be extracted from {self.name}.",
                    "yellow",
                )

                while True:
                    try:
                        power_string = input(
                            "Pls. enter incident power (e.g. xxx u(m)wW:"
                        )

                        if "uW" in power_string:
                            self.power = [
                                float(aa) for aa in power_string.split() if aa.isdigit()
                            ]
                            self.power = self.power[0] * u.microwatt
                            break

                        elif "mW" in power_string:
                            self.power = [
                                float(aa) for aa in power_string.split() if aa.isdigit()
                            ]
                            self.power = self.power[0] * u.milliwatt
                            break

                        elif "na" in power_string:
                            self.power = power_string

                        else:
                            raise ValueError

                    except ValueError:
                        printmd(" Unit (u/mW) was missing!", "red")
                        continue

                    else:
                        break

            return self.power
            self.fname_corr_flag = 1

        def _pressure():
            """
            Returns the sample pressure condition for the given experiment.
            The pressure is extracted from the file name (if extractable!),
            otherwise asks the user to  input it manually. \n
            """

            if "mbar" in self.nameCorrect5:
                subString = "mbar"
                result = [aa for aa in self.name_split_list1 if subString in aa]
                self.pressure = float(result[0][0:-6]) * u.millibar

                if result[0] == "mbar":
                    pressure_index = self.name_split_list1.index("mbar")
                    self.pressure = (
                        float(self.name_split_list1[power_index - 1]) * u.millibar
                    )

                else:
                    pass

            elif "atm" in self.nameCorrect5:
                self.pressure = 1.013e3 * u.mbar

            else:
                print("\n")
                printmd(
                    f"The pressure info is either mising or could not be extracted {self.name}.",
                    "yellow",
                )

                while True:
                    try:
                        pressure_string = input(
                            "Pls. enter pressure (e.g. x.xE(-)x mbar):"
                        )

                        if "mbar" in pressure_string:
                            self.pressure = [
                                float(aa)
                                for aa in pressure_string.split()
                                if aa.isdigit()
                            ]
                            self.pressure = self.pressure[0] * u.mbar
                            break

                        elif "na" in pressure_string:
                            self.pressure = pressure_string

                        else:
                            raise ValueError

                    except ValueError:
                        printmd(" Unit (mbar) was missing!", "red")
                        continue

                    else:
                        break

            return self.pressure
            self.fname_corr_flag = 1

        # ----------------------------------------------------------------------------------------------------------------- #

        """
            with open(file, mode="rb") as self.spe: # The "with open()" method ensures the closing of file at the end after               reading the respective .SPE binary (rb: read binary) files. This is a better way to work with the files,                     restricting writing to a file outside the "with" method. Nevertheless, I had to use the below method so that the             "standard_info" function can read the file and display the informarion only once. Otherwise, it will display the             standard experimental settings for each file.
        """

        # Open files with "read binary" mode and return a corresponding file object. 'file' is a path-like object (string) giving the pathname.
        self.spe = open(file, mode="rb") 

        def _data_dimension():
            """
            Returns the dimensions of X and Y axis pixels.

            REMARK: The manual is misleading for datatypes. It says short(int8) datatype but actually needs int16.
            I had to convert the dimensions into int64 type, otherwise it does not work when I want to exctract the actulal               data bytes using x_Dim*y_Dim in the "_experiment_data()" method.
            """

            self.x_Dim = np.int64(
                read_at(
                    self.spe,
                    self.X_PIXELS,
                    1,
                    np.int16,
                )[0]
            )  # no. of pixels on x axis
            self.y_Dim = np.int64(
                read_at(
                    self.spe,
                    self.Y_PIXELS,
                    1,
                    np.int16,
                )[0]
            )  # no. of pixels on y axis

            return self.y_Dim, self.x_Dim

        def _exposure_time():
            self.seconds = read_at(
                self.spe,
                self.EXPOSURE_TIME,
                1,
                np.float32,
            )[0]

            return self.seconds

        def _accumulations():  # The data type should be different for both the cases. Nonetheless,an experiment with >32767 accumulation is needed to check the code.
            self.accumulations = read_at(
                self.spe,
                self.ACCUMULATIONS1,
                1,
                np.int16,
            )[0]
            if self.accumulations == -1:  # if > 32767
                self.accumulations = read_at(
                    self.spe,
                    self.ACCUMULATIONS2,
                    1,
                    np.int16,
                )[0]

            return self.accumulations

        # No idea what does it represent?
        def _no_frames():  # Number of image frames in the SPE file
            self.frames = read_at(
                self.spe,
                self.NUM_FRAMES,
                1,
                np.int16,
            )[0]

            return self.frames

        def _step_glue():
            self.step_Glue = read_at(
                self.spe,
                self.STEP_GLUE_FLAG,
                1,
                np.int8,
            )[0]
            self.sgString = "No"

            if self.step_Glue == 1:
                self.sgString = "Yes"
            else:
                pass

            return self.sgString

        def _axes():
            """
            Axis calibration with a polynomial function of the order of 1 (for Step&Glue) and 2 (for non SG);
            meaning the non-SG x-axis is calibrated with a quadratic equation (coff1.x^2 + coff2.x^1 + coff2.x^0),
            while the SG x-axis is claibrated with a linear eq.
            """
            # ------------------------------------------ x-axis calibration -------------------------------------------#

            # It always shows 532 nm. Perhaps the 532 nm laser was used for x-calibration. during installation of the CCD.
            self.laser = read_at(
                self.spe,
                self.X_LASER_POSITION,
                1,
                np.double,
            )[0]
            # self.ROI = self.read_at(self.spe, self.NUM_ROI, 10, np.int8)[0]  # No. of ROIs; NOT WORKING!

            xPolyOrder = read_at(
                self.spe,
                self.X_POLYNOMIAL_ORDER,
                1,
                np.int8,
            )[0]
            # No. of total coeff=6; no. of non-zero coeff.= polyOrder+1
            xPolyCoeff = np.float32(
                read_at(
                    self.spe,
                    self.X_POLYNOMIAL_COEFFICIENT,
                    6,
                    np.double,
                )
            )
            # print("X Polynomial coeff:", xPolyCoeff)
            xCalibValid = read_at(
                self.spe,
                self.X_CALIBRATION_VALID,
                1,
                np.int8,
            )[0]

            if xCalibValid:  # Slice only non-zero array elements.
                xCoeff = xPolyCoeff[: xPolyOrder + 1]  
                # reverse coefficients to use numpy polyval; <object_name>[<start_index>, <stop_index>, <step>]
                xCoeffRvrs = np.array(xCoeff[::-1])
                # 1d array of a pixel range; from 1 (to usually 1100)
                xPixels = np.linspace(1, self.x_Dim, self.x_Dim)  
                # If p is of length N, then function returns: p[0]*x**(N-1) + p[1]*x**(N-2) + ... + p[N-2]*x + p[N-1]
                # x_axis = np.polyval(xCoeffRvrs, xPixels,)
                # The 1d (or 0d) ndarrays can be a problem. There are many ways of adding a 2nd dimension like,"[None,:]                       syntax, reshape  ndmin=2, np.atleast_2d, np.expand_dims".
                # Store in a 2D array (matrix) with a shape of (n_rows (i.e. 1), n_cols (i.e. xPixels)).
                # This way it's more convinient to think and also for further analysis (e.g. for reversing data points.)
                # self.x_Axis = np.reshape(x_axis, (1, np.shape(x_axis)[0]))
                self.x_Axis = np.array(
                    np.polyval(
                        xCoeffRvrs,
                        xPixels,
                    ),
                    ndmin=2,
                )

            else:
                self.x_Axis = np.array(
                    np.linspace(start=1, stop=self.x_Dim, counts=self.x_Dim), ndmin=2
                )

            # Selecting first and last elements of the 1D X-axis matrix or a 2D array.
            self.xaxis_range = [self.x_Axis[0][0], self.x_Axis[0][-1]]

            if self.step_Glue == 1:
                self.resolution = read_at(
                    self.spe,
                    self.WAVELENGTH_RESOLUTION,
                    1,
                    np.float32,
                )[0]
                self.overlap = read_at(
                    self.spe,
                    self.WAVELENGTH_OVERLAP,
                    1,
                    np.float32,
                )[0]

            else:
                self.resolution = self.x_Axis[0][1] - self.x_Axis[0][0]
                self.overlap = "None"

            # -------------------------------------------- y-axis calibration ---------------------------------------------#
            yPolyOrder = read_at(
                self.spe,
                self.Y_POLYNOMIAL_ORDER,
                1,
                np.int8,
            )[0]
            yPolyCoeff = np.float32(
                read_at(
                    self.spe,
                    self.Y_POLYNOMIAL_COEFFICIENT,
                    6,
                    np.double,
                )
            )

            yCalibValid = read_at(
                self.spe,
                self.Y_CALIBRATION_VALID,
                1,
                np.int8,
            )[0]

            if yCalibValid:
                yCoeff = yPolyCoeff[: yPolyOrder + 1]
                yCoeffRvrs = np.array(
                    yCoeff[::-1]
                )  # reverse coefficients to use numpy polyval
                yPixels = np.linspace(1, self.y_Dim, self.y_Dim)
                self.y_Axis = np.array(
                    np.polyval(
                        yCoeffRvrs,
                        yPixels,
                    ),
                    ndmin=2,
                )

            else:
                self.y_Axis = np.array(np.linspace(1, self.y_Dim, self.y_Dim), ndmin=2)

            if len(self.y_Axis) > 1:
                self.yaxis_range = [
                    self.y_Axis[0],
                    self.y_Axis[-1],
                ]

            else:
                self.yaxis_range = "N.A. (Binned)"

            return (
                self.x_Axis,
                self.xaxis_range,
                self.y_Axis,
                self.yaxis_range,
                self.resolution,
                self.overlap,
            )

        def _experiment_data():
            """Reads data (with correct data conversion type for different scenarios) from an imported .SPE file from all                    .SPE files and reshapes it into the [[no. of frames]*[x axis dim.]*[y axis dim.]] format. If any anomalies                    happen while reading the data such as occuring of any "0" then raises an exception.

            NOTE: If there are more no. of frames (>1) then the code needs to be modified."""

            dataArray = []

            for frame in range(self.frames):
                # The final extracted data should be of float type otherwise performing operations on data would be cumbersome.
                # Special care is required when the data is converted using the given datataype!
                # When SG is used, the overlapping data is of float type in the original .SPE files.
                if self.step_Glue == 1:
                    rawData = read_at(
                        self.spe, self.DATA_START, self.x_Dim * self.y_Dim, np.float32
                    )

                # This will work only when the data value is <32767.
                else:
                    rawData = np.array(
                        read_at(
                            self.spe,
                            self.DATA_START,
                            self.x_Dim * self.y_Dim,
                            np.uint16,
                        ),
                        dtype=np.float32,
                    )

                    # uint32 is only required when atleast one data value is >32767. Do not use it otherwise.
                    if (
                        rawData[1] == 0
                    ):  # One special case is solved. May be there are more!
                        rawData = np.array(
                            read_at(
                                self.spe,
                                self.DATA_START,
                                self.x_Dim * self.y_Dim,
                                np.uint32,
                            ),
                            dtype=np.float32,
                        )

                    else:
                        pass

                dataArray.append(rawData)

            self.data = np.asanyarray(dataArray).reshape(
                self.frames,
                self.y_Dim,
                self.x_Dim,
            )

            if self.data.any() == 0:
                raise Exception(
                    "Data value 0 found! Either the {} file is corrupted or something went wrong with the data extraction. Please check the datatype to read binary data correctly.".format(
                        self.name
                    )
                )

            else:
                pass

            return self.data

        def _grating():
            self.grooves = read_at(
                self.spe,
                self.GRATING_GROOVES,
                1,
                np.float32,
            )[0]
            if self.grooves != 300:
                printmd(
                    f"The {self.name} experiment was performed with {self.grooves}/mm grating."
                )

            else:
                pass

            return self.grooves

        _step_glue()
        _no_frames()
        _experiment_temp()
        _excitation_source()
        _incident_power()
        _exposure_time()
        _accumulations()
        _data_dimension()
        _axes()
        _experiment_data()
        _pressure()
        _grating()

        spedict = {
            "Name": self.name,
            "01) Data": self.data,
            "02) Wavelength range [nm]": [
                round(
                    self.xaxis_range[0],
                    2,
                ),
                round(
                    self.xaxis_range[1],
                    2,
                ),
            ],
            "03) Temperature": self.temperature,
            "04) Incident power": self.power,
            "05) Exposure [sec]": self.seconds,
            "06) Accumulation(s)": self.accumulations,
            "07) Excitation source": self.ext_source_string,
            #             "08) Strip [px]": self.yaxis_range,
            "08) Dimension [px]": (
                self.y_Dim,
                self.x_Dim,
            ),
            "09) Resolution [nm]": self.resolution,
            # "10) No. of ROIs": self.ROI,
            "10) Overlap [nm]": self.overlap,
            "11) Pressure": self.pressure,
            "12) Grating [grooves/mm]": self.grooves,
        }

        return spedict

    # --------------------------------------------- Standard Information ---------------------------------------------------#
    #  Needs to be printed only once since all the experiments were performed under the same conditions.
    def _standard_info(self):
        print("\n")
        display(MD("**Standard information**"))

        def _messege():
            if self.fname_corr_flag == 1:
                print("\n")
                messeges = {
                    "m0": "**Prefered name : sample_experiment_(img)_acc*expo(s)_source(cw/pulsed/wl)_power_temp_pressure \nE.g. : WSe2/WS2_PL_2Dimg_10x120s_532nm(cw)_675µW_10K_5.2E-5mb**"
                }

                printmd(messeges["m0"], "green")

        # No idea what does it represent?
        def _no_frames():  # Number of image frames in the SPE file
            frames = read_at(
                self.spe,
                self.NUM_FRAMES,
                1,
                np.int16,
            )[0]
            printmd(f"No. of frames : {frames}", "white")

        def _experiment_date_time():
            rawdate = read_at(
                self.spe,
                self.EXPERIMENT_DATE,
                10,
                np.int8,
            )
            dateString = ""
            aa = 0
            for ch in rawdate:
                dateString += chr(ch)
                aa += 1
                if aa % 2 == 0 and aa < 4:
                    dateString += "-"  # To make it more readable
                elif aa == 5:
                    dateString += "-"
                else:
                    pass
            date = dateString

            rawtime = read_at(
                self.spe,
                self.EXPERIMENT_TIME_LOCAL,
                6,
                np.int8,
            )
            timeString = ""
            bb = 0
            for ch in rawtime:
                timeString += chr(ch)
                bb += 1

                if bb % 2 == 0 and bb < 5:
                    timeString += ":"  # To make it more readable
            time = timeString

        #             display(MD("Date and Time: {}, {}".format(date, time)))

        def _detector_temp():
            temperature = read_at(
                self.spe,
                self.DETECTOR_TEMP,
                1,
                np.float32,
            )[0]
            if temperature > -120:
                printmd(
                    f"The {self.spe} experiment was recorded at {temperature} °C chip temp. However, the chip temp. should be max. -120 °C. to improve the SNR for Priceton acton SPE2300 detector."
                )

            else:
                printmd(f"Detector temp. : {temperature} °C")

        def _intensified_gain():
            iGain = read_at(
                self.spe,
                self.PIMAX_GAIN,
                1,
                np.int8,
            )[0]
            printmd(f"Intensified gain : {iGain}")

        def _versions():
            package = read_at(
                self.spe,
                self.SOFTWARE_PACKAGE,
                1,
                np.uint8,
            )[0]
            controller = read_at(
                self.spe,
                self.CONTROLLER_VERSION,
                16,
                np.int8,
            )[0]
            header = read_at(
                self.spe,
                self.HEADER_VERSION,
                1,
                np.float32,
            )[0]
            software = read_at(
                self.spe,
                self.SOFTWARE_VERSION,
                16,
                np.int8,
            )
            softwareString = ""

            for ch in software:
                softwareString += chr(ch)

            display(MD("**Version information :**"))
            printmd(f"Software package : {package}")
            printmd(f"Controller : {controller}")
            printmd(f"Header : {header}".format())
            printmd(f"Software : {softwareString}")

        def _data_corrections():
            display(MD("**Data correction:**"))
            background = read_at(
                self.spe,
                self.BG_CORRECTION,
                1,
                np.int8,
            )[0]
            bgString = "No"
            if background == 1:
                bgString = "Yes"
            printmd(f"Backgroung applied : {bgString}")

            flatField = read_at(
                self.spe,
                self.FLATFIELD_CORRECTION,
                1,
                np.int8,
            )[0]
            ffString = "No"
            if flatField == 1:
                ffString = "Yes"
            printmd(f"Flat field applied : {ffString}")

            cosmic = read_at(
                self.spe,
                self.COSMIC_CORRECTION,
                1,
                np.int8,
            )[0]
            cosmicString = "No"
            if cosmic == 1:
                cosmicString = "Yes"

            printmd(f"Cosmic applied : {cosmicString}")

            if cosmic == 1:
                cosmicCorrType = read_at(
                    self.spe,
                    self.COSMIC_CORR_TYPE,
                    1,
                    np.int8,
                )
                cosmicThreshold = read_at(
                    self.spe,
                    self.COSMIC_THRESHOLD,
                    1,
                    np.int8,
                )
                printmd(f"Cosmic type : {cosmicCorrType}")
                printmd(f"Cosmic threshold : {cosmicThreshold}")

        #             self.xGain = self.read_at(self.spe, GAIN, 1, np.int8)[0]

        #             comments = self.read_at(self.spe, FILE_COMMENTS, 400, np.int8)
        #             commentString=""
        #             for ch in comments:
        #                 commentString += chr(ch)
        #             print("Comments:", commentString)

        _no_frames()
        _detector_temp()
        _intensified_gain()
        _versions()
        _data_corrections()
        _messege()

## Reflection contrast analysis

In [ ]:
fId = File()
fileInfo = DF()  # Create a Dataframe

spe_files = []
data_All = []
xaxis_All = []
yaxis_All = []

for files in fId.fnames:
    # Append the output of spe_read() (spedict) into a dataframe.
    fileInfo = fileInfo.append(
        fId._spe_read(files),
        ignore_index=True,
    )
    data_All.append(fId.data)
    xaxis_All.append(fId.x_Axis)
    yaxis_All.append(fId.y_Axis)
    spe_files.append(fId.spe)  # Append file objects to close them later on!


# The Pandas frame append function looses the order of dictionary columns and sorts them alphabatically. No idea how to fix it!
# Display the file info in a tabular dataframe after transposing it.
display(fileInfo.set_index("Name").T)  

fId._standard_info()  # call the function to present the standard info.


file_close(spe_files)

### Axis and Data

In [ ]:
# u.enable_contexts('sp')

# ------------------------------------------------------ X - axis ------------------------------------------------------#
wavelength = Q_(xaxis_All, "nm")  # equvalent to xaxis_All * u.nm

# print(wavelength.units)
# print(wavelength.magnitude)


with u.context("sp"):
    energy = wavelength.to("eV")

"""
    If the X-axis is wavelength then there is no need to reverse the data.
    When it's converted into the energy axis then it makes more sense to start the axis with ascending values
"""

energy = data_reverse(data=energy.magnitude)  # np array with a unit cannot be flipped.
energy = np.round(energy, 5)
energy = Q_(energy, "eV")  # reassign the unit.
# print(energy.units)
print("\n")


# -------------------------------------------------------- Data -------------------------------------------------------- #
data_All = data_reverse(data=data_All)
rfl_sam = data_All[0][0]  # Select the data only
rfl_sub = data_All[1][0]

### Region selection

In [ ]:
no_regns = 2
stpts = [40, 21]
endpts = [49, 34]

printmd("**Reflection_sample info**")
rfl_sam_region, rfl_sam_region_binned = region_bin(
    data=rfl_sam, no_regns=no_regns, stpts=stpts, endpts=endpts
)

print("\n\n")

printmd("**Reflection_substrate info**")
rfl_sub_region, rfl_sub_region_binned = region_bin(
    data=rfl_sub, no_regns=no_regns, stpts=stpts, endpts=endpts
)

### Reflection contrast of selected (binned) regions

In [ ]:
printmd("**RC of selected regions**")
rflcnt_regn = reflection_contrast(rfl_sam_region, rfl_sub_region)

print("\n\n")

printmd("**RC of binned selected regions**")
rflcnt_regn_binned = reflection_contrast(rfl_sam_region_binned, rfl_sub_region_binned)

### Differential reflection contrast

In [ ]:
rflcnt_regn_derv = differentiation(rflcnt_regn, energy)
rflcnt_regn_derv_unit = rflcnt_regn_derv[0].units  # list object has no unit.
print("\n")

rflcnt_regn_binned_derv = differentiation(rflcnt_regn_binned, energy)
rflcnt_regn_binned_derv_unit = rflcnt_regn_binned_derv[
    0
].units  # list object has no unit.
print(rflcnt_regn_binned_derv_unit)

### Moving window filter

In [ ]:
window_size = 5

rflcnt_regn_derv_flatMvgAvg = mvg_window_fltr(
    data=rflcnt_regn_derv[0].magnitude, window_type=("flat", 5), mode="reflect"
)
# rflcnt_regn_derv_flatMvgAvg = mvg_window_fltr(data = data, window_type = ('flat', 5), mode='reflect')
# window = signal.boxcar(51)
print(rflcnt_regn_derv_flatMvgAvg)
# print(window, len(window))

## (Interactive) matplotlib ploting

In [ ]:
import matplotlib as mpl
from cycler import cycler
from matplotlib import cm
from matplotlib import gridspec as gridspec
from matplotlib import (
    image as mpimg,  # to dispay images (limited formats are supported)
)
from matplotlib import pyplot as plt
from matplotlib import rc, ticker
from matplotlib.pyplot import imshow as imshow
from matplotlib.ticker import (  # for labels formattings
    FormatStrFormatter,
    MultipleLocator,
)
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

%matplotlib widget
#%matplotlib inline

### Global style setting

In [ ]:
def _plot_params():

    # ---------------------------- Font ---------------------------- #
    # plt.rcParams['font.family'] = "serif" # by default Halvatic font is used
    # plt.rcParams['font.serif'] = "Helvetica"
    plt.rcParams["font.family"] = "sans-serif"
    plt.rcParams["font.weight"] = 200
    plt.rcParams["text.usetex"] = True  # use latex for all text handling.
    plt.rcParams["axes.prop_cycle"] = cycler(color="bgrcmyk")
    plt.rcParams["font.size"] = 12

    # ---------------------------- Figure ----------------------------#
    plt.rcParams["figure.figsize"] = [3.0, 3.0]
    # plt.rcParams['figure.dpi'] = 80
    plt.rcParams["savefig.dpi"] = 600
    plt.rcParams["savefig.format"] = "svg"  # eps
    plt.rcParams["savefig.transparent"] = True
    plt.rcParams["svg.fonttype"] = "none"
    # plt.rcParams['figure.edgecolor'] = 'black'

    # ---------------------------- LINEWIDTH ---------------------------- #
    plt.rcParams["lines.linewidth"] = 1.3
    plt.rcParams["lines.scale_dashes"] = True

    # ---------------------------- LEGEND ---------------------------- #
    plt.rcParams["legend.loc"] = "best"
    plt.rcParams["legend.fontsize"] = plt.rcParams["font.size"]
    plt.rcParams["legend.framealpha"] = None
    plt.rcParams["legend.borderpad"] = 0.2
    plt.rcParams["legend.facecolor"] = "inherit"
    # plt.rcParams['legend.edgecolor'] = 0.0

    # ---------------------------- AXES ---------------------------- #
    plt.rcParams["axes.titleweight"] = 300
    plt.rcParams["axes.labelsize"] = plt.rcParams["font.size"]
    plt.rcParams["axes.labelweight"] = 300
    plt.rcParams["axes.labelweight"] = "normal"
    plt.rcParams["axes.linewidth"] = 1.5

    plt.rcParams["font.weight"] = "normal"

    # ---------------------------- TICKS ----------------------------#
    plt.rcParams["xtick.top"] = True
    plt.rcParams["xtick.major.size"] = 7
    plt.rcParams["xtick.minor.size"] = 4
    plt.rcParams["xtick.major.width"] = 1.5
    plt.rcParams["xtick.minor.width"] = 1
    plt.rcParams["xtick.labelsize"] = plt.rcParams["font.size"]
    plt.rcParams["xtick.direction"] = "in"
    plt.rcParams["xtick.minor.visible"] = True

    plt.rcParams["ytick.right"] = True
    plt.rcParams["ytick.major.size"] = 7
    plt.rcParams["ytick.minor.size"] = 4
    plt.rcParams["ytick.major.width"] = 1.5
    plt.rcParams["ytick.minor.width"] = 1
    plt.rcParams["ytick.labelsize"] = plt.rcParams["font.size"]
    plt.rcParams["ytick.direction"] = "in"
    plt.rcParams["ytick.minor.visible"] = True

    # ---------------------------- IMAGES ----------------------------#
    plt.rcParams["image.aspect"] = "equal"
    plt.rcParams["image.cmap"] = "inferno"
    plt.rcParams["image.origin"] = "upper"
    plt.rcParams[
        "image.composite_image"
    ] = True  # saving a figure as a vector graphics file

    plt.style.use("seaborn-white")


_plot_params()

plt.close()

# sns.set_context('poster')  #Everything is larger

## Fitting

### 1. Gaussian fitting

$$P(x) = \frac{1}{\sigma\sqrt{2\pi}}e^{-(x-\mu)^2/(2\sigma^2)}$$

```python
def gauss(x, aplt, mean, std_dev, offset):
    sum = offset
    for i in range(len(ampl)):
        sum = sum + ampl[i]*np.exp(-(x - mean[i])**2 / (2*std_dev[i]**2))
 
    return sum
```

In [ ]:
# Gaussian fit for a single peak
def gaus_1(x, a_1, x_1, sigma_1, offset):
    return a_1 * np.exp(-((x - x_1) ** 2) / (2 * sigma_1 ** 2)) + offset


# Gaussian fit for two individual peaks
def gaus_2(x, a_1, x_1, sigma_1, offset, a_2, x_2, sigma_2):
    return (
        a_1 * np.exp(-((x - x_1) ** 2) / (2 * sigma_1 ** 2))
        + a_2 * np.exp(-((x - x_2) ** 2) / (2 * sigma_2 ** 2))
        + offset
    )

### 2. Pearson IV fitting

$A(E)\ =\ \frac{|(\Gamma(m+\frac{\nu}{2}i)|^2}{\alpha \beta (m - \frac{1}{2}, \frac{1}{2})} [1 + (\frac{E - \lambda}{\alpha})^2]^{-m} \cdot exp[-\nu \cdot arctan(\frac{E - \lambda}{\alpha})] $ 

The first factor of the Pearson IV distribution is the normalizing constant, which involves the complex Gamma function ($\Gamma$) and the Beta function ($\beta$). The parameter $\lambda$ is the localisation parameter and represents the center of the peak, while $\alpha$ > 0 is a scale parameter, which defines the width of the function. The parameter $\nu$ describes the asymmetry or skewness of the function and $m$ > 1$/$2 the general shape. For $\nu\ =$ 0 the function is the symmetric Student’s t distribution. For $m$ = 1 the function is a skewed version of a Lorentzian, while for $m$ $\rightarrow$ $\infty$ the function becomes a skewed Gaussian. The scale and the shape parameters also influence the maximum of the distribution, which is given by

In [ ]:
def pearsonIV_1(energy, alpha, Lambda, m, nu, offset):
    num1 = np.square(
        np.absolute(sc.special.gamma(m + (nu / 2) * j) / (sc.special.gamma(m)))
    )
    den1 = alpha * sc.special.beta(m - (1 / 2), 1 / 2)
    fact1 = num1 / den1
    term1 = np.power(1 + np.square(((energy - Lambda) / alpha)), -m)
    term2 = np.exp(-nu * (np.arctan((energy - Lambda) / alpha)))

    return fact1 * term1 * term2 + offset


def pearsonIV_2(energy, alpha1, Lambda1, m1, nu1, alpha2, Lambda2, m2, nu2, offset):
    return (
        pearsonIV_1(energy, alpha1, Lambda1, m1, nu1, 0)
        + pearsonIV_1(energy, alpha2, Lambda2, m2, nu2, 0)
        + offset
    )